In [118]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import datetime
import numpy as np
import psd_tool
import data_loader
from field_models import model
import matplotlib.pyplot as plt
from matplotlib import cm
import spacepy.time
import fokker_planck
from matplotlib import colors
import plot_tools


SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import importlib
importlib.reload(psd_tool)
importlib.reload(data_loader)
importlib.reload(fokker_planck)
importlib.reload(plot_tools)

%matplotlib qt
plt.ioff() #This is awesome and delays the drawing until plt.show(). Thank god



In [119]:
#Data Properties

start = datetime.datetime(year = 2013,
                            month = 3,
                            day = 1) - datetime.timedelta(hours=8.75)

end = datetime.datetime(year = 2013, 
                        month = 4, 
                        day = 1) + datetime.timedelta(hours=8.75)

satellite = "A"
chosen_mu = 1000
chosen_k = 0.07
model = model.TS04D


#Simulation Properties

#These are the min and max Lstar for the actual data... this must be satisfied: (lstar_min < lstar_of_min_BC < lstar_of_max_BC < lstar_max)
lstar_min = 2.0
lstar_max = 6.0

#These are the lstar of the max and min boundary conditions
Lstar_of_min_BC = 2.75
Lstar_of_max_BC = 5.0

#These are the steps used in the simulation. dL should satisfy: (lstar_of_max_BC - lstar_of_min_BC) / dL = [SOME INTEGER]
dL = 0.10
dt = datetime.timedelta(seconds = 0.5)

#What paper to get the diffusion coefficients from
sim_diffusion_coefficient_type = fokker_planck.DIFFUSION_COEFFICIENT_TYPE_1D.RBSP_SCALED

In [120]:
#Load Data

refs = data_loader.load_psd(satellite = satellite, field_model = model, start = start, end = end)

selected_refs = psd_tool.select_mu_and_k_from_psd(refs = refs, chosen_mu = chosen_mu, chosen_k = chosen_k)

epoch = selected_refs["EPOCH"]
epoch_JD = np.array(spacepy.time.Ticktock(epoch, "UTC").JD)
Lstar = selected_refs["L_STAR"] 
L = selected_refs["L"]
PSD = selected_refs["PSD"]
in_out = selected_refs["IN_OUT"]
orbit_number = selected_refs["ORBIT_NUMBER"]

not_nan = np.isfinite(PSD)
epoch = epoch[not_nan]
epoch_JD = epoch_JD[not_nan]
Lstar = Lstar[not_nan]
L = L[not_nan]
PSD = PSD[not_nan]
in_out = in_out[not_nan]
orbit_number = orbit_number[not_nan]

OMNI_1hr_refs = data_loader.load_omni_data_1hour_res(start, end + datetime.timedelta(days = 1))
OMNI_epoch_JD = np.array(spacepy.time.Ticktock(OMNI_1hr_refs["EPOCH"], "UTC").JD)
OMNI_Kp = OMNI_1hr_refs["Kp"]
OMNI_Kp[OMNI_Kp < 0] = np.NaN
OMNI_not_nan = np.isfinite(OMNI_epoch_JD) & np.isfinite(OMNI_Kp)
OMNI_epoch_JD = OMNI_epoch_JD[OMNI_not_nan]
OMNI_Kp = OMNI_Kp[OMNI_not_nan]

2013-02-01 00:00:00
Loading : PSD_201302_A_TS04D.npz
2013-03-01 00:00:00
Loading : PSD_201303_A_TS04D.npz
2013-04-01 00:00:00
Loading : PSD_201304_A_TS04D.npz
Time taken for loop: 6.486746311187744
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20130101_v01.cdf
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20130701_v01.cdf


In [121]:
#Get Initial Orbit, BCs, and clean the data prior to simulation. All variables with "sim" prefix are simulation ready.

unique_orbits = np.sort(np.unique(orbit_number))[1:]

initial_orbit = (orbit_number == unique_orbits[0])
initial_PSD = PSD[initial_orbit]
initial_Lstar = Lstar[initial_orbit]
initial_epoch_JD = epoch_JD[initial_orbit]

binned_initial_Lstar, binned_initial_PSD = psd_tool.bin_radial_profile(LSTAR = initial_Lstar,
                                                                       PSD = initial_PSD,
                                                                       LSTAR_MIN = lstar_min,
                                                                       LSTAR_MAX = lstar_max,
                                                                       dL = dL)

where_binned_initial_PSD_not_nan = np.isfinite(binned_initial_PSD)
sim_Lstar_grid = np.arange(start = Lstar_of_min_BC, stop = Lstar_of_max_BC + dL, step = dL)

sim_initial_PSD = np.interp(x = sim_Lstar_grid,
                            xp = binned_initial_Lstar[where_binned_initial_PSD_not_nan],
                            fp = binned_initial_PSD[where_binned_initial_PSD_not_nan],
                            left = np.NaN,
                            right = np.NaN)


if np.isnan(sim_initial_PSD[0]) or np.isnan(sim_initial_PSD[-1]):
    
    raise Exception(f"Tried to do a simulation between: Lstar_of_min_BC = {Lstar_of_min_BC} and Lstar_of_max_BC = {Lstar_of_max_BC}, but the first orbit doesn't have valid PSD at the boundaries!")

#Get Boundary Conditions

JD_of_orbits = [np.nanmean(initial_epoch_JD)]
PSD_of_min_BC = [sim_initial_PSD[0]]
PSD_of_max_BC = [sim_initial_PSD[-1]]

for orbit in unique_orbits[1:]:

    full_orbit = (orbit_number == orbit)
    PSD_orbit = PSD[full_orbit]
    Lstar_orbit = Lstar[full_orbit]
    epoch_JD_orbit = epoch_JD[full_orbit]
    
    points_at_min_BC = ((Lstar_of_min_BC - dL / 2.0) < Lstar_orbit) & (Lstar_orbit <= (Lstar_of_min_BC + dL / 2.0))
    points_at_max_BC = ((Lstar_of_max_BC - dL / 2.0) < Lstar_orbit) & (Lstar_orbit <= (Lstar_of_max_BC + dL / 2.0))
    
    JD_of_orbits.append(np.nanmean(epoch_JD_orbit))
    PSD_of_min_BC.append(np.nanmean(PSD_orbit[points_at_min_BC]))
    PSD_of_max_BC.append(np.nanmean(PSD_orbit[points_at_max_BC]))

JD_of_orbits = np.asarray(JD_of_orbits)
PSD_of_min_BC = np.asarray(PSD_of_min_BC)
PSD_of_max_BC = np.asarray(PSD_of_max_BC)


now_and_dt_later_in_JD = spacepy.time.Ticktock([datetime.datetime.now(), (datetime.datetime.now() + dt)], "UTC").JD
dt_JD = now_and_dt_later_in_JD[1] - now_and_dt_later_in_JD[0]



JD_grid = np.arange(JD_of_orbits[0], JD_of_orbits[-1] + dt_JD, dt_JD)

where_min_BC_is_not_nan = np.isfinite(PSD_of_min_BC)
interpolated_PSD_at_min_BC = np.interp(x = JD_grid,
                                       xp = JD_of_orbits[where_min_BC_is_not_nan],
                                       fp = PSD_of_min_BC[where_min_BC_is_not_nan],
                                       left = np.NaN,
                                       right = np.NaN)



where_max_BC_is_not_nan = np.isfinite(PSD_of_max_BC)
interpolated_PSD_at_max_BC = np.interp(x = JD_grid,
                                       xp = JD_of_orbits[where_max_BC_is_not_nan],
                                       fp = PSD_of_max_BC[where_max_BC_is_not_nan],
                                       left = np.NaN,
                                       right = np.NaN)

sim_Kp = np.interp(x = JD_grid,
                   xp = OMNI_epoch_JD, 
                   fp = OMNI_Kp,
                   left = np.NaN,
                   right = np.NaN)

if np.any(np.isnan(sim_Kp)):
    raise Exception("Some of the Kp values were NaN! This breaks everything at this point, and I was too lazy to fix it until it became an issue!")

#Notice this was carefully programmed so the NaNs only appear on the right hand side of the array, this gets the intersection. The timestep between consecutive points should still be dt_JD.
orbits_that_have_points_at_both_BCs = np.isfinite(interpolated_PSD_at_min_BC) & np.isfinite(interpolated_PSD_at_max_BC)

sim_JD_grid = JD_grid[orbits_that_have_points_at_both_BCs]
sim_PSD_at_min_BC = interpolated_PSD_at_min_BC[orbits_that_have_points_at_both_BCs]
sim_PSD_at_max_BC = interpolated_PSD_at_max_BC[orbits_that_have_points_at_both_BCs]

C:\Users\brofe\AppData\Local\Temp\ipykernel_24268\2285501111.py:48: RuntimeWarning: Mean of empty slice
  PSD_of_max_BC.append(np.nanmean(PSD_orbit[points_at_max_BC]))


In [122]:
print(f"Calculating diffusion coefficients with: {sim_diffusion_coefficient_type.name}")

D_ll, dD_ll_dL = fokker_planck.calculate_diffusion_coefficients(L = sim_Lstar_grid,
                                                                Kp = np.expand_dims(sim_Kp, axis = 1),
                                                                type = sim_diffusion_coefficient_type)

print(f"Running diffusion simulation...")


Lstar_S, f_S, T_S = fokker_planck.fokker_planck_1D_simulation(iterations_between_saves = 100000,
                                                              dt = dt_JD, 
                                                              dL = dL,
                                                              L = sim_Lstar_grid,
                                                              f = sim_initial_PSD,
                                                              min_BCs = sim_PSD_at_min_BC,
                                                              max_BCs = sim_PSD_at_max_BC,
                                                              D_ll = D_ll,
                                                              dD_ll_dL = dD_ll_dL)

Epoch_S = spacepy.time.Ticktock(sim_JD_grid[0] + (T_S * dt_JD), "JD").UTC

print(f"Total iterations: {len(sim_PSD_at_min_BC)}")

Calculating diffusion coefficients with: RBSP_SCALED
Running diffusion simulation...
Total iterations: 5356755


In [123]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)
fig.set_size_inches(14, 10)
fig.suptitle(f"RBSP-{satellite.upper()}, $\mu$ = {chosen_mu} (MeV / G), K = {chosen_k} ($R_E$$G^{{1/2}}$), {model.name}")

#First we plot the actual data!

start_of_orbits = []

for orbit in unique_orbits:
    
    full_orbit = (orbit_number == orbit)
    
    if np.any(np.nonzero(full_orbit)[0]):
        
        start_of_orbit = epoch[np.nonzero(full_orbit)[0][0]]
        
        binned_Lstar, binned_PSD = psd_tool.bin_radial_profile(LSTAR = Lstar[full_orbit],
                                                               PSD = PSD[full_orbit],
                                                               LSTAR_MIN = lstar_min,
                                                               LSTAR_MAX = lstar_max,
                                                               dL = dL)
        
        psd_tool.plot_radial_profile(LSTAR = binned_Lstar,
                                     PSD = binned_PSD,
                                     IS_INBOUND = True,
                                     START_OF_ORBIT = start_of_orbit,
                                     AXIS = axs[0])
        
        start_of_orbits.append(start_of_orbit)


axs[0].set_axisbelow(True)
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlabel("L*")
axs[0].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[0].set_xlim((lstar_min, lstar_max))

handles, labels = axs[0].get_legend_handles_labels()
order = np.argsort(start_of_orbits)
line_colors = plt.get_cmap("viridis", len(order)) # Initialize holder for trajectories
j = 0
for i in order:
    handles[i].set_color(line_colors(j))
    j += 1

axs[0].legend([handles[i] for i in order], [labels[i] for i in order])
axs[0].set_title(rf"Observed")

# Now we plot the simulation!
for T in range(f_S.shape[0]):
    
    psd_tool.plot_radial_profile(LSTAR = Lstar_S,
                                 PSD = f_S[T, :],
                                 IS_INBOUND = True,
                                 START_OF_ORBIT = Epoch_S[T],
                                 AXIS = axs[1])
    
axs[1].set_axisbelow(True)
axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].set_xlabel("L*")
axs[1].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[1].set_xlim((lstar_min, lstar_max))

handles, labels = axs[1].get_legend_handles_labels()
order = np.argsort([i for i in range(f_S.shape[0])])
line_colors = plt.get_cmap("viridis", len(order)) # Initialize holder for trajectories
j = 0
for i in order:
    handles[i].set_color(line_colors(j))
    j += 1

axs[1].legend([handles[i] for i in order], [labels[i] for i in order])
axs[1].set_title(rf"1-D Diffusion, $D_{{LL}}$[{sim_diffusion_coefficient_type.name}]")

plt.show()

<>:3: SyntaxWarning: invalid escape sequence '\m'


In [125]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)
fig.set_size_inches(14, 10)

axs[0].set_title(rf"1-D Diffusion, $D_{{LL}}$[{sim_diffusion_coefficient_type.name}]")
axs[0].set_ylabel("L*")
axs[0].set_ylim(Lstar_S[0], Lstar_S[-1])
axs[0].set_xlim(Epoch_S[0], Epoch_S[-1])
simulated_cmap = axs[0].imshow(f_S.T,
                 extent=[Epoch_S[0], Epoch_S[-1], Lstar_S[0], Lstar_S[-1]],
                 norm=colors.LogNorm(vmin = np.amin(f_S[np.isfinite(f_S)]), vmax = np.amax(f_S[np.isfinite(f_S)])),
                 origin="lower",
                 aspect="auto",
                 interpolation="none",
                 cmap=cm.get_cmap("viridis"))

#To be honest I have no idea how this actually works anymore. Matplotlib is bad and they make it difficult to position the colorbars.
axins = inset_axes(
    axs[0],
    width="1%",  # width: 5% of parent_bbox width
    height="100%",  # height: 50%
    loc="lower left",
    bbox_to_anchor=(1.01, 0, 1, 1),
    bbox_transform=axs[0].transAxes,
    borderpad=0
)

cbar = plt.colorbar(simulated_cmap, cax=axins)    
cbar.set_label(r"PSD $(c/MeV/cm)^{3}$", loc="center", labelpad=25, rotation=270)

outer_belt = outer_belt = (Lstar_S[0] < Lstar) & (Lstar < Lstar_S[-1])

plot_tools.plot_l_vs_time_log_colors(x = epoch[outer_belt], 
                                     y = Lstar[outer_belt], 
                                     c = PSD[outer_belt], 
                                     cbar_label = r"PSD $(c/MeV/cm)^{3}$", 
                                     vmin = np.amin(f_S[np.isfinite(f_S)]), 
                                     vmax = np.amax(f_S[np.isfinite(f_S)]), 
                                     axis = axs[1])

axs[1].set_title(f"Observed, RBSP-{satellite}")
axs[1].set_ylabel("L*")
axs[1].set_xlabel("Time")
axs[1].set_ylim(Lstar_S[0], Lstar_S[-1])
axs[1].set_xlim(Epoch_S[0], Epoch_S[-1])

plt.show()


C:\Users\brofe\AppData\Local\Temp\ipykernel_24268\187137372.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap("viridis"))
